In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
df=pd.read_csv('Fraud_check.csv')
df.head()

<IPython.core.display.Javascript object>

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [40]:
df['Undergrad']=df['Undergrad'].astype(object) # the datatype of columns (undergrad,urban and marital_status) were of 
                                               #int datatype so label encoder wasn't working on it. so converted the datatype 
                                               #for those column to object.

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [42]:
df['Marital.Status']=df['Marital.Status'].astype(object)
df['Urban']=df['Urban'].astype(object)


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [44]:
print(df['Undergrad'].value_counts())
print(df['Marital.Status'].value_counts())
print(df['Urban'].value_counts())

YES    312
NO     288
Name: Undergrad, dtype: int64
Single      217
Married     194
Divorced    189
Name: Marital.Status, dtype: int64
YES    302
NO     298
Name: Urban, dtype: int64


# Label encoding for object datatype column

In [45]:
lab_enco = LabelEncoder()
df['Undergrad']=lab_enco.fit_transform(df['Undergrad'])
df['Urban']=lab_enco.fit_transform(df['Urban'])
df['Marital.Status']=lab_enco.fit_transform(df['Marital.Status'])
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,0,2,68833,50047,10,1
1,1,0,33700,134075,18,1
2,0,1,36925,160205,30,1
3,1,2,50190,193264,15,1
4,0,1,81002,27533,28,0
...,...,...,...,...,...,...
595,1,0,76340,39492,7,1
596,1,0,69967,55369,2,1
597,0,0,47334,154058,0,1
598,1,1,98592,180083,17,0


In [46]:
print(df['Undergrad'].value_counts())
print(df['Marital.Status'].value_counts())
print(df['Urban'].value_counts())

1    312
0    288
Name: Undergrad, dtype: int64
2    217
1    194
0    189
Name: Marital.Status, dtype: int64
1    302
0    298
Name: Urban, dtype: int64


# adding categorical column for income status

In [47]:
income_status = []

for i in df['Taxable.Income']:
    if i<=30000:
        income_status.append('Risky')
    else:
        income_status.append('Good')
df['income_status'] = income_status
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income_status
0,0,2,68833,50047,10,1,Good
1,1,0,33700,134075,18,1,Good
2,0,1,36925,160205,30,1,Good
3,1,2,50190,193264,15,1,Good
4,0,1,81002,27533,28,0,Good
...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good
596,1,0,69967,55369,2,1,Good
597,0,0,47334,154058,0,1,Good
598,1,1,98592,180083,17,0,Good


In [48]:
df['income_status'].value_counts()

Good     476
Risky    124
Name: income_status, dtype: int64

# applying RF

In [53]:
df
x=df.iloc[:,[0,1,3,4,5]]
y=df.iloc[:,-1]
print(x)
print(y)

     Undergrad  Marital.Status  City.Population  Work.Experience  Urban
0            0               2            50047               10      1
1            1               0           134075               18      1
2            0               1           160205               30      1
3            1               2           193264               15      1
4            0               1            27533               28      0
..         ...             ...              ...              ...    ...
595          1               0            39492                7      1
596          1               0            55369                2      1
597          0               0           154058                0      1
598          1               1           180083               17      0
599          0               0           158137               16      0

[600 rows x 5 columns]
0      Good
1      Good
2      Good
3      Good
4      Good
       ... 
595    Good
596    Good
597    Good
598 

In [64]:
kfold = KFold(n_splits=10,random_state=7,shuffle=True)
model = RandomForestClassifier(n_estimators=500,max_features=4)
result = cross_val_score(model,x,y,cv=kfold)
result.mean()

0.7433333333333334

In [65]:
model.fit(x,y)
df['pred_loan_status'] = model.predict(x)
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income_status,pred_loan_status
0,0,2,68833,50047,10,1,Good,Good
1,1,0,33700,134075,18,1,Good,Good
2,0,1,36925,160205,30,1,Good,Good
3,1,2,50190,193264,15,1,Good,Good
4,0,1,81002,27533,28,0,Good,Good
...,...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good,Good
596,1,0,69967,55369,2,1,Good,Good
597,0,0,47334,154058,0,1,Good,Good
598,1,1,98592,180083,17,0,Good,Good


# Applying bagging

In [67]:
dt =DecisionTreeClassifier()
model1 = BaggingClassifier(base_estimator=dt,n_estimators=500,random_state=42)
result1 = cross_val_score(model1,x,y,cv=kfold)
result1.mean()

0.7416666666666666

In [68]:
model1.fit(x,y)
df['pred_bagging'] = model1.predict(x)
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income_status,pred_loan_status,pred_bagging
0,0,2,68833,50047,10,1,Good,Good,Good
1,1,0,33700,134075,18,1,Good,Good,Good
2,0,1,36925,160205,30,1,Good,Good,Good
3,1,2,50190,193264,15,1,Good,Good,Good
4,0,1,81002,27533,28,0,Good,Good,Good
...,...,...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good,Good,Good
596,1,0,69967,55369,2,1,Good,Good,Good
597,0,0,47334,154058,0,1,Good,Good,Good
598,1,1,98592,180083,17,0,Good,Good,Good


# Applying Adaboost

In [69]:
model2 = AdaBoostClassifier(n_estimators=500,random_state=42)
result2 = cross_val_score(model1,x,y,cv=kfold)
result2.mean()

0.7416666666666666

In [70]:
model2.fit(x,y)
df['pred_boosting'] = model2.predict(x)
df

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban,income_status,pred_loan_status,pred_bagging,pred_boosting
0,0,2,68833,50047,10,1,Good,Good,Good,Good
1,1,0,33700,134075,18,1,Good,Good,Good,Good
2,0,1,36925,160205,30,1,Good,Good,Good,Good
3,1,2,50190,193264,15,1,Good,Good,Good,Good
4,0,1,81002,27533,28,0,Good,Good,Good,Good
...,...,...,...,...,...,...,...,...,...,...
595,1,0,76340,39492,7,1,Good,Good,Good,Good
596,1,0,69967,55369,2,1,Good,Good,Good,Good
597,0,0,47334,154058,0,1,Good,Good,Good,Good
598,1,1,98592,180083,17,0,Good,Good,Good,Good
